<a href="https://colab.research.google.com/github/tilakparth/Automatic-Image-Captioning/blob/main/Automatic_Image_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
import pickle
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# from keras.utils import to_categorical

In [11]:
def load_document(filename):
	# opening the file as read only
	file = open(filename, 'r')
	# reading all text
	text = file.read()
	# closing the file
	file.close()
	return text

filename = "/content/drive/MyDrive/flicker8k/Flickr_Data/Flickr_TextData/Flickr8k.token.txt"
# loading descriptions
doc = load_document(filename)
print(doc[:300])

1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg#3	A little girl climbing the s


In [12]:
def load_descriptions(doc):
	mapping = dict()
	# processing lines
	for line in doc.split('\n'):
		# spliting line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# taking the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# extracting filename from image id
		image_id = image_id.split('.')[0]
		# converting description tokens back to string
		image_desc = ' '.join(image_desc)
		# creating the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# storing description
		mapping[image_id].append(image_desc)
	return mapping

# parsing descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

Loaded: 8092 


In [13]:
list(descriptions.keys())[0:10]

['1000268201_693b08cb0e',
 '1001773457_577c3a7d70',
 '1002674143_1b742ab4b8',
 '1003163366_44323f5815',
 '1007129816_e794419615',
 '1007320043_627395c3d8',
 '1009434119_febe49276a',
 '1012212859_01547e3f17',
 '1015118661_980735411b',
 '1015584366_dfcec3c85a']

In [14]:
descriptions['1007320043_627395c3d8']

['A child playing on a rope net .',
 'A little girl climbing on red roping .',
 'A little girl in pink climbs a rope bridge at the park .',
 'A small child grips onto the red ropes at the playground .',
 'The small child climbs on a red ropes on a playground .']

In [15]:
def clean_descriptions(descriptions):
	# preparing translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			# tokenizing
			desc = desc.split()
			# converting to lower case
			desc = [word.lower() for word in desc]
			# removing punctuation from each token
			desc = [w.translate(table) for w in desc]
			# removing hanging 's' and 'a'
			desc = [word for word in desc if len(word)>1]
			# removing tokens with numbers in them
			desc = [word for word in desc if word.isalpha()]
			# storing as string
			desc_list[i] =  ' '.join(desc)

# cleaning descriptions
clean_descriptions(descriptions)

In [16]:

descriptions['1007320043_627395c3d8']

['child playing on rope net',
 'little girl climbing on red roping',
 'little girl in pink climbs rope bridge at the park',
 'small child grips onto the red ropes at the playground',
 'the small child climbs on red ropes on playground']

In [17]:
# converting the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# building a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

# summarizing vocabulary
vocabulary = to_vocabulary(descriptions)
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 8763


In [18]:
# saving descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_descriptions(descriptions, 'descriptions.txt')

In [39]:
# loading a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_document(filename)
	dataset = list()
	# processing line by line
	for line in doc.split('\n'):
		# skiping empty lines
		if len(line) < 1:
			continue
		# geting the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# loading training dataset (6K)
filename = '/content/drive/MyDrive/flicker8k/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt'
train = load_set(filename)
# print(train)
# print('Dataset: %d' % len(train))

In [20]:
# Below path contains all the images
images = '/content/drive/MyDrive/flicker8k/Flickr_Data/Images/'
# Creating a list of all image names in the directory
img = glob.glob(images + '*?.jpg')

In [21]:
# Below file conatains the names of images to be used in train data
train_images_file = '/content/drive/MyDrive/flicker8k/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt'
# Reading the train image names in a set
train_images = set(open(train_images_file, 'r').read().strip().split('\n'))
# Creating a list of all the training images with their full path names

train_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in train_images: # Checking if the image belongs to training set
        train_img.append(i) # Adding it to the list of train images

In [22]:
# Below file conatains the names of images to be used in test data
test_images_file = '/content/drive/MyDrive/flicker8k/Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt'
# Read the validation image names in a set# Read the test image names in a set
test_images = set(open(test_images_file, 'r').read().strip().split('\n'))

# Create a list of all the test images with their full path names
test_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in test_images: # Check if the image belongs to test set
        test_img.append(i) # Add it to the list of test images

In [24]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_document(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))

Descriptions: train=6000


In [29]:
def preprocess(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = image.load_img(image_path, target_size=(299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [30]:
# Load the inception v3 model
model = InceptionV3(weights='imagenet')

96124928/96112376 [==============================] - 2s 0us/step


In [31]:
# Create a new model, by removing the last layer (output layer) from the inception v3
model_new = Model(model.input, model.layers[-2].output)

In [32]:
# Function to encode a given image into a vector of size (2048, )
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

In [33]:
# Call the funtion to encode all the train images
# This will take a while on CPU - Execute this only once
start = time()
encoding_train = {}
for img in train_img:
    encoding_train[img[len(images):]] = encode(img)
print("Time taken in seconds =", time()-start)

Time taken in seconds = 3597.372329711914


In [36]:
# Save the bottleneck train features to disk
with open("/content/drive/MyDrive/flicker8k/Flickr_Data/Pickle/encoded_train_images.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_train, encoded_pickle)

In [37]:
# Call the funtion to encode all the test images - Execute this only once
start = time()
encoding_test = {}
for img in test_img:
    encoding_test[img[len(images):]] = encode(img)
print("Time taken in seconds =", time()-start)

Time taken in seconds = 580.6892600059509


In [40]:
# Save the bottleneck test features to disk
with open("/content/drive/MyDrive/flicker8k/Flickr_Data/Pickle/encoded_test_images.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_test, encoded_pickle)